In [37]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from datasets import load_dataset
import nltk
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
from sklearn.preprocessing import FunctionTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

### Cargamos datos

In [38]:


# Con esto lo bajamos de este repositorio centralizado
ds = load_dataset("ag_news")

vect = CountVectorizer()
# Si quisieramos que considere en el diccionario secuencia de un caracter (como la palabra "y", hay que modificar el parámetro token_pattern al instanciar CountVectorizer
# vect = CountVectorizer(token_pattern=r'(?u)\b\w+\b')
vect.fit(ds["train"]["text"])

KeyboardInterrupt: 

In [ ]:
#matriz=vect.transform(ds["train"]["text"]).todense()

In [ ]:
# Convertimos a pandas

id2label = ds["train"].features["label"].names

df_train = ds["train"].to_pandas()

df_test = ds["test"].to_pandas()

In [39]:
# Tomamos una proporción de la muestra de entrenamiento para que el entrenamiento sea más eficiente computacionalmente
muestra_fraccion = 0.01

df_train_muestra= df_train.sample(frac=muestra_fraccion, random_state=42)
X_train=df_train_muestra["text"]
y_train=df_train_muestra["label"]

In [ ]:

# Obtener el vocabulario generado por el CountVectorizer
vocabulario = vect.get_feature_names_out()

# Mostrar las primeras 50 palabras del vocabulario
print(vocabulario[:50])

# Si deseas conocer el tamaño total del vocabulario
print("\nTamaño total del vocabulario:", len(vocabulario))

['00' '000' '000th' '01' '02' '037' '038' '04' '05' '06' '08' '09' '10'
 '100' '100m' '100th' '101' '1010' '1040' '105' '106' '108' '10k' '10m'
 '10th' '11' '110' '114' '116' '118' '11th' '12' '123' '128mb' '12th' '13'
 '130' '130b' '133' '135' '137th' '139m' '13b' '14' '140' '14012' '141'
 '142' '146' '147']

Tamaño total del vocabulario: 8560


In [ ]:
for text in df_train_muestra['text'].sample(10):
    print(text)
    print('-'*80)

UN Mission Sends Military Team to Investigate Fighting in Eastern &lt;b&gt;...&lt;/b&gt; Following reports of fighting between different factions of former members of the Armed Forces of the Democratic Republic of the Congo (FARDC) and weapons distribution to civilians in North Kivu province, the United Nations mission today said it sent a 
--------------------------------------------------------------------------------
Jaguar's Coventry plant to close Jaguar's historic Brown's Lane factory in Coventry is to close, Ford has confirmed, with 400 jobs expected to be lost.
--------------------------------------------------------------------------------
Brown Says He May Never Get Over Athens (AP) AP - Larry Brown didn't have much time to enjoy his first NBA title. After coaching the Detroit Pistons to a surprising victory over the Los Angeles Lakers, Brown focused all his energy on getting the U.S. basketball team ready for the Olympics. Much to his dismay, the Americans came home with jus

In [ ]:
#Generamos un pipeline con todos los pasos del pre procesamiento
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators)
pipeline = Pipeline([
    ('vectorizador', CountVectorizer(token_pattern=r'\b[a-zA-Z]+\b', strip_accents="ascii", stop_words="english", analyzer='word'))
])

rf.fit(X_train, y_train)
X_train = pipeline.fit_transform(df_train_muestra['text'])
vocabulario_alfabetico = pipeline.named_steps['vectorizador'].get_feature_names_out()
print(vocabulario_alfabetico[:50])


['aapl' 'ababa' 'abandon' 'abandoned' 'abbey' 'abc' 'abdicate' 'abdomen'
 'abducted' 'abduction' 'able' 'abn' 'aboard' 'aboriginal' 'abounds'
 'abroad' 'absence' 'abtahi' 'abu' 'abuse' 'abuses' 'abusing' 'academic'
 'academics' 'accelerates' 'accept' 'acceptable' 'accepted' 'accepts'
 'access' 'accessible' 'acclaim' 'acclaimed' 'accomplished' 'accord'
 'according' 'accountancy' 'accounting' 'accounts' 'accusations' 'accuse'
 'accused' 'accuser' 'accuses' 'accusing' 'ace' 'aceh' 'acer' 'achieve'
 'achievements']


In [44]:
#Generamos un pipeline con todos los pasos del pre procesamiento


rf = RandomForestClassifier()
"""
vect=CountVectorizer(token_pattern=r'\b[a-zA-Z]+\b', strip_accents="ascii", stop_words="english", analyzer='word')
"""
vect=CountVectorizer(token_pattern=r'\b[a-zA-Z]+\b', strip_accents="ascii", analyzer='word')
pipeline = Pipeline(steps=[
    ('vect', vect),
    ('rf', rf)
])

"""param_grid = {
    "vect__ngram_range": [(x, x) for x in [1,2,3,4]],  # Suponiendo que quieras ajustar ngrams
    "rf__n_estimators": [10,20,30]
}"""

param_grid = {
    'vect__max_df': [0.5, 0.75, 1.0],
    'vect__min_df': [1, 2, 3, 4],
    'vect__max_features': [None, 5000, 10000, 15000],
    'vect__ngram_range': [(1, 1), (1, 2), (2, 2)],
    'vect__stop_words': [None, 'english'],
    #n estimators en el numeros de arboles b
    "rf__n_estimators": [10,20,30,50,75,100],
    #max features es la cantidad de variables en cada nodo entre la que elije la de menor error
    "rf_"

}


search = GridSearchCV(pipeline, param_grid, n_jobs=2)
search.fit(X_train, y_train)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)


Best parameter (CV score=0.751):
{'rf__n_estimators': 30, 'vect__max_df': 0.75, 'vect__max_features': 10000, 'vect__min_df': 2, 'vect__ngram_range': (1, 1), 'vect__stop_words': 'english'}


In [ ]:
rf.fit(X_train, y_train)
X_train_opt = pipeline.fit_transform(df_train_muestra['text'])
vocabulario_alfabetico = pipeline.named_steps['vectorizador'].get_feature_names_out()
print(vocabulario_alfabetico[:50])


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Define a pipeline to search for the best combination of PCA truncation
# and classifier regularization.
pca = PCA()
# Define a Standard Scaler to normalize inputs
scaler = StandardScaler()

# set the tolerance to a large value to make the example faster
logistic = LogisticRegression(max_iter=10000, tol=0.1)
pipe = Pipeline(steps=[("scaler", scaler), ("pca", pca), ("logistic", logistic)])

X_digits, y_digits = datasets.load_digits(return_X_y=True)
# Parameters of pipelines can be set using '__' separated parameter names:
param_grid = {
    "pca__n_components": [5, 15, 30, 45, 60],
    "logistic__C": np.logspace(-4, 4, 4),
}
search = GridSearchCV(pipe, param_grid, n_jobs=2)
search.fit(X_digits, y_digits)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

# Plot the PCA spectrum
pca.fit(X_digits)

fig, (ax0, ax1) = plt.subplots(nrows=2, sharex=True, figsize=(6, 6))
ax0.plot(
    np.arange(1, pca.n_components_ + 1), pca.explained_variance_ratio_, "+", linewidth=2
)
ax0.set_ylabel("PCA explained variance ratio")

ax0.axvline(
    search.best_estimator_.named_steps["pca"].n_components,
    linestyle=":",
    label="n_components chosen",
)
ax0.legend(prop=dict(size=12))

RandomForestClassifier()